Analysis of CG-MD simulations of self-assembling peptides system In the present tutorial, the authors reported the workflow with the main functions for analyzing CG-MD simulations. The analyzed simulation consists of 2 AQP structure embedded within a neuronal membrane.In each box, the authors reported the main instructions and the required packages for running the analysis.

In [1]:
import morphoscanner as ms
import MDAnalysis as mda
import numpy as np
from timeit import default_timer as timer

/Users/federicofontana/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [2]:
#Write the full trajectory just for the peptides
trj_gro = '/Users/federicofontana/Documents/CNTE/Membrane-Peptide-Interactions/LKLK-Membrane-Peptide-Interactions/eq_LKLK_Membrane-F-out.gro'
trj_xtc = '/Users/federicofontana/Documents/CNTE/Membrane-Peptide-Interactions/LKLK-Membrane-Peptide-Interactions/production-1/prd-LKLK-Fibril-1000ns-out.xtc'
u = mda.Universe(trj_gro,trj_xtc)
prot = u.select_atoms('index 1-1880') #previous index 29082-31482 and name BB
prot_traj_name = "/Users/federicofontana/Documents/CNTE/Membrane-Peptide-Interactions/LKLK-Membrane-Peptide-Interactions/production-1/prd-LKLK-Fibril-1000ns-out-AQP.xtc"
with mda.Writer(prot_traj_name, prot.n_atoms) as W:
    for ts in u.trajectory:
        W.write(prot)
#Write the first frame of the trajectory
u_min = mda.Universe(trj_gro,trj_gro)
prot_min = u_min.select_atoms('index 1-1880')
prot_min.write("/Users/federicofontana/Documents/CNTE/Membrane-Peptide-Interactions/LKLK-Membrane-Peptide-Interactions/production-1/eq_LKLK_Membrane-F-out-AQP.gro")

/Users/federicofontana/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/federicofontana/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/federicofontana/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/federicofontana/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to 

Locate the files of the initial configuration of the system and trajectory

In [3]:
_gro ='/Users/federicofontana/Documents/CNTE/Membrane-Peptide-Interactions/LKLK-Membrane-Peptide-Interactions/production-1/eq_LKLK_Membrane-F-out-AQP.gro'
_xtc ='/Users/federicofontana/Documents/CNTE/Membrane-Peptide-Interactions/LKLK-Membrane-Peptide-Interactions/production-1/prd-LKLK-Fibril-1000ns-out-AQP.xtc'

# Single trajectory analysis

The class morphoscanner.trajectory.trajectory() instantiate the class instance trajectory(), that is responsible to perform the analysis on a MD trajectory. The instance trajectory() contains the results of the analysis, and the capability to visualize the computed data.

The trajectory() class need few arguments to be instantiated and start the analysis:

_sys_config : str

file path of the initial configuration of the system (.gro file for GROMACS) _sys_traj : str

file path of the trajectory file (or files) (.xtc or .trr in GROMACS) select : list(str), optional

The parameter select let you choose which atoms to parse. These are the atoms that will be used to perform the analysis. Atoms definitions are taken from the Martini FF v2.2 and are found in the module ms.molnames.constituents. Default value is select = ['aminoacids'], that will select all the aminoacids alpha-carbon atoms. select = ['aminoacids'] will select the following atoms:

ms.molnames.aminoacids_list\ \ ['GLY', 'ALA', 'CYS', 'VAL', 'LEU', 'ILE', 'MET', 'PRO', 'ASN', 'GLN', 'ASP', 'ASP0', 'GLU', 'GLU0', 'THR', 'SER', 'LYS', 'LYS0', 'ARG', 'ARG0', 'HIS', 'HISH', 'PHE', 'TYR', 'TRP', 'BTNr4']

'aminoacids' can be changed with one of the ms.molnames.costituents.keys() to select other atom types

In [4]:
a = ms.trajectory.trajectory(_gro,_xtc, select=['aminoacids'])

In your trajectory there are 2001 frames.

Length: 223, Peptides: 4


/Users/federicofontana/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:215: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "


The exploratory step is needed to initializate a trajectory. It gives the possibility to give a look to the trajectory before to start the analysis.

In [5]:
a.explore()

Exploration of frame 0 done.



# Parse frames of the trajectory

The input MD trajectory can be analyzed on each frame (timestep). Calling compose_database() enable the frames sampling and parsing.

sampling_interval is an int.\ Each frame of the trajectory has an index = i that is an int in F = [0, number_of_frame).\ If i / sampling_interval = 0, the frame is sampled for subsequent analysis. \ The sampled frames are the frames of which the index i is a multiple of sampling_interval.

In [6]:
a.compose_database(sampling_interval=20)

100%|██████████| 100/100 [00:00<00:00, 275.02it/s]


# Analysis of the sampled frames

After the frame sampling and data retrieval each sampled frame of the system is analyzed with our algorithm, to search for the emergence of complex structures that match a beta sheets pattern.

The method analyze_inLoop() takes as parameters:

threshold : float, optional\ The default is 5.\ threshold is the longest distance at which two points i,j are considered in contact.\ This means that if ed(i,j) < threshold, the points i and j are contacting.\ ed is the euclidean distance. threshold_multiplier : float, optional The default is 1.5. threshold_multiplier is a factor used to multiply the calculated threshold distance for contact recognition.\ The parameter threshold_multipier is overwritten if threshold is passed as parameters. device : str, optional\ The device on which the data are saved and the analysis is computed.\ The option are: cpu, to perform parallelization on cpu, cuda, to performe parallelization on CUDA compatible device, usually Nvidia GPUs.\ \ The default is cpu.

In [7]:
start = timer()
a.analyze_inLoop(threshold=5.1, threshold_multiplier=1.5, device='cpu')

processing started...

Analyzing frame n°  0
Time to compute distance is:  0.039487917000002426 seconds.


100%|██████████| 2/2 [00:00<00:00, 17084.74it/s]


Time to denoise:  1.3720661659999962 seconds.
Finished analysis of frame n° 0
Time needed to analyze frame 0 was 1.433978 seconds.

Analyzing frame n°  20
Time to compute distance is:  0.012630708999978424 seconds.


0it [00:00, ?it/s]


Time to denoise:  1.2222869999999944 seconds.
Finished analysis of frame n° 20
Time needed to analyze frame 20 was 1.235242 seconds.

Analyzing frame n°  40
Time to compute distance is:  0.01301929199999563 seconds.


...

Time to denoise:  1.0116721659999826 seconds.
Finished analysis of frame n° 1960
Time needed to analyze frame 1960 was 1.030215 seconds.

Analyzing frame n°  1980
Time to compute distance is:  0.01283587500000749 seconds.


0it [00:00, ?it/s]


Time to denoise:  0.9576676249999991 seconds.
Finished analysis of frame n° 1980
Time needed to analyze frame 1980 was 0.970673 seconds.

Analyzing frame n°  2000
Time to compute distance is:  0.012475417000018751 seconds.


100%|██████████| 1/1 [00:00<00:00, 17623.13it/s]

Time to denoise:  0.9845799169999623 seconds.
Finished analysis of frame n° 2000
Time needed to analyze frame 2000 was 0.999696 seconds.

Total time to analyze dataset was 101.009686 seconds
.


The analysis() workflow consist of the following steps:

threshold calculation, only if threshold=None frame's atoms coordinates retrival from the coordinate's database pairwise distance maps calculation contact maps calculation, using the threshold calculated above pattern recognition, analyze contact for matching with beta-sheet pattern compose a graph of contacting peptide matching the beta-sheet pattern search for isolated communities (cluster of peptides) in the graph Each step will be here described in its logic and implementation.

threshold calculation The threshold is the distance used to discriminate between atoms that are involved in an interaction.\ The distance between points i and j is d[i][j] = sqrt(|i^2 - j^2|).\ If d < threshold, points i and j are defined as contacting.

The threshold is an important parameter, because subsequent analysis steps depends on the contact found between peptides.

Default threshold is 5 (Angstrom).

If threshold == None, the threshold is calculated by morphoscanner. The calculate threshold distance is the median distance between all the contiguous apha-carbon median distance of each peptides aminoacid in a frame, multiplied by a threshold_multiplier.

threshold is calculated as:

median(median(ed(p[c], p[c+1]) for c in [0, len(p)) for each p in f).\ \ Where T is the set of the sampled frames,\ f is a frame in T,\ p is a peptide in f,\ c is an alpha-carbon in p,\ len(p) is the number of c in p,\ ed is the euclidean distance,\ median is the median value.

The result of this calculation in our mesurements are between 3.3 and 3.5 Angstrom.

Inter alpha-carbon distance can be calculated and retrieved using the function backend.distance_tensor.sample_intrapeptide_distance().

The above function can be used after compose_database. It gets as argument the trajectory() object on which the trajectory was composed, and samples, that are the number of frames to sample to get the inter alpha-carbon distance. samples = 1 by default.

In the following cell we calculate the inter alpha-carbon distance for the trajectory() object a instantiated above, on 5 sampled frame.

# Recover Data after the analysis

The method get_data() run a set of other methods that recover data from the analysis. Additional class attributes are created.\ The obtained data are saved in pandas.DataFrame, reachable from a.database.\ In a.database there is a row for each sampled and analyzed timestep (or frame) of the trajectory. For each sampled timestep, the following data are saved in separate columns:

n° of peptides in macroaggragate:\ is an ordered descending list() in which each element is the number of peptide found in a peptide cluster parallel:\ is an int, the number of parallel contact in that timestep antiparallel:\ is an int, the number of antiparallel contact in that timestep n° of macroaggregate:\ is an int, the number of clusters in the frame. Is equal to len(n° of peptides in macroaggragate). There are then a number of other columns, in which the label in an int from 0 to N that depends on the number N of peptides cluster found in that frame. If clusters are found, the corresponding columns contains a dict() with the key:value:

parallel : int antiparallel : int In which the int value define the number of parallel or antiparallel contacts on that cluster.

In [8]:
a.get_data()

# Data Visualization

The instance of the class morphoscanner.tajectory.trajectory() has a set of methods to visualize the data obtained from the analysis.

plot_frame_aggregate(frame=int) Plot the frame with a color code that identify the sense of the majority of contacts in an cluster.

Grey: no contact, Green: majority of parallel contacts, Blue: majority of antparallel contacts, Yellow: equal number of parallel and antiparallel contacts

In [ ]:
ms.analysis.plot_aggregates_average([a], label='a')

In [ ]:
ms.analysis.plot_contacts_average([a], label='a')

In [ ]:
ms.analysis.plot_beta_average([a], label='a')

In [ ]:
ms.analysis.plot_3d_parallel_average([a])

In [ ]:
ms.analysis.plot_3d_antiparallel_positive_average([a])

In [ ]:
ms.analysis.plot_3d_antiparallel_negative_average([a])

In [ ]:
#import nglview as nv
#from MDAnalysis import transformations as trans
#u = mda.Universe(_gro, _xtc)
#prot = u.select_atoms('protein')
#t = nv.MDAnalysisTrajectory(prot)
#w = nv.NGLWidget(t)
#w

# Calculate helix score for a single sampled frame

In [9]:
a.calculate_helix_score_for_frame(0)

The results of the helix score calulation are saved in the result object of the selected frame.

In [10]:
a.frames[0].results.helix_score

{0: {'pace_helix': 3.999999994181296,
  'n_carbonalpha': 116,
  'perc_alpha': 52.01793721973094,
  'length_beta': 4.135853499733592,
  'n_carbonbeta': 24,
  'perc_beta': 10.762331838565023,
  'explored_alpha': array([  3.,   5.,   6.,   7.,   8.,  10.,  12.,  14.,  16.,  17.,  18.,
          19.,  20.,  21.,  29.,  30.,  36.,  39.,  40.,  41.,  42.,  43.,
          44.,  46.,  47.,  48.,  53.,  54.,  55.,  57.,  59.,  60.,  63.,
          68.,  69.,  72.,  73.,  76.,  79.,  80.,  82.,  83.,  84.,  85.,
          86.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98., 100.,
         102., 103., 104., 105., 106., 107., 108., 113., 122., 123., 124.,
         126., 127., 130., 133., 134., 135., 136., 138., 139., 140., 141.,
         142., 143., 144., 145., 146., 147., 148., 153., 159., 165., 167.,
         168., 169., 170., 171., 173., 175., 182., 184., 185., 186., 190.,
         192., 193., 194., 198., 200., 201., 202., 205., 207., 210., 211.,
         215., 217., 219., 220., 221., 222

# Calculate the helix score for each sampled frame of the trajectory

In [11]:
a.helix_score()

In [12]:
a.frames[100].results.helix_score

{0: {'pace_helix': 4.989900031239072,
  'n_carbonalpha': 100,
  'perc_alpha': 44.843049327354265,
  'length_beta': 5.10458427406272,
  'n_carbonbeta': 29,
  'perc_beta': 13.004484304932735,
  'explored_alpha': array([  7.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  23.,
          29.,  32.,  33.,  34.,  35.,  39.,  44.,  47.,  51.,  54.,  55.,
          56.,  59.,  62.,  68.,  69.,  70.,  71.,  72.,  73.,  75.,  78.,
          79.,  80.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  91.,  92.,
          94.,  95.,  97.,  98., 100., 103., 105., 106., 107., 110., 113.,
         114., 115., 122., 123., 126., 128., 130., 132., 133., 134., 135.,
         136., 137., 138., 139., 142., 144., 146., 148., 150., 161., 163.,
         167., 169., 171., 175., 176., 177., 184., 185., 186., 190., 191.,
         195., 198., 201., 202., 203., 211., 212., 213., 218., 219., 220.,
         221.]),
  'explored_beta': array([ 40.,  41.,  42.,  43.,  45.,  50.,  61.,  64.,  89.,  90., 109.,
        